Hotel Recommendation System

Bizim yaklaşımımız müşteri yorumlarına ve derecelendirmelerine dayalı bir öneri sistemi oluşturmaktır.Kullanıcı puanlarına göre bir öneri sistemi oluşturacağım için burada doğal dil işleme kullanacağım. 

In [57]:
# Kullanacağım dataset.

import kagglehub

# Download latest version
path = kagglehub.dataset_download("jiashenliu/515k-hotel-reviews-data-in-europe")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Asus\.cache\kagglehub\datasets\jiashenliu\515k-hotel-reviews-data-in-europe\versions\1


In [58]:
# Veri setini okuyalım.

import pandas as pd
df=pd.read_csv("C:/Users/Asus/.cache/kagglehub/datasets/jiashenliu/515k-hotel-reviews-data-in-europe/versions/1/Hotel_Reviews.csv")
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [59]:
# Hotel_Address sütununun son kelimesi ülke adı oluyor.Ülke adını ayrı bir sütun haline getirelim.
# Aynı zamanda United Kingdom'u UK olarak değiştirelim.(Tek kelime olsun)

df['Hotel_Address'] = df['Hotel_Address'].str.replace("United Kingdom", "UK")
df['Country'] = df['Hotel_Address'].str.split(' ').str[-1]
print(df['Country'].value_counts())

Country
UK             262301
Spain           60149
France          59928
Netherlands     57214
Austria         38939
Italy           37207
Name: count, dtype: int64


In [60]:
# DataFrame'deki güncel satır ve sütun sayısını görelim.
df.shape

(515738, 18)

In [61]:
# Kaç tane tekrar eden veri var?
print(df.duplicated().sum())  

526


In [62]:
# Veri setinde 526 tane tekrar eden satır var.Bunları kaldıralım.
# Ardından df.shape ile temizleme sonrası DataFrame'deki güncel satır ve sütun sayısını görelim.

df = df.drop_duplicates()
print(f"Yeni veri seti boyutu: {df.shape}")

Yeni veri seti boyutu: (515212, 18)


In [63]:
# Eksik/hatalı veriler var mı?
print(df.isnull().sum())

Hotel_Address                                    0
Additional_Number_of_Scoring                     0
Review_Date                                      0
Average_Score                                    0
Hotel_Name                                       0
Reviewer_Nationality                             0
Negative_Review                                  0
Review_Total_Negative_Word_Counts                0
Total_Number_of_Reviews                          0
Positive_Review                                  0
Review_Total_Positive_Word_Counts                0
Total_Number_of_Reviews_Reviewer_Has_Given       0
Reviewer_Score                                   0
Tags                                             0
days_since_review                                0
lat                                           3268
lng                                           3268
Country                                          0
dtype: int64


In [64]:
# lat ve lng sütunlarında eksik veriler var.Bu sütunları çıkaracağım.
# Modelin doğruluğunu etkileyip etkilemediğini daha sonra test edeceğim.Gerekirse geri ekleyeceğim.

df.drop(['lat', 'lng'], axis=1, inplace=True)

# Verinin yeni halini kontrol edelim
print(df.shape)

(515212, 16)


In [65]:
#Modelin doğruluğunu etkilemeyecek diğer sütunları da çıkaracağım.

df.drop(['Reviewer_Nationality','days_since_review',],axis=1,inplace=True)
print(df.shape)

(515212, 14)


In [66]:
# Dataset'te 6 farklı ülke var.
print(df['Country'].unique())

['Netherlands' 'UK' 'France' 'Spain' 'Italy' 'Austria']


In [67]:
# Review_Date sütunu üzerinde işlemler yapacağım.
# datetime formatına dönüştüreceğim. 
# errors='coerce' parametresi kullanıldığında, geçersiz veya hatalı tarih formatları olan hücreler
# NaT (Not a Time) değerine dönüştürülür. 
# Bu sayede veri setindeki tarih formatları tutarsız olsa bile hatalar düzgün şekilde ele alınır.

df['Review_Date'] = pd.to_datetime(df['Review_Date'], errors='coerce')
print(df['Review_Date'].head())


0   2017-08-03
1   2017-08-03
2   2017-07-31
3   2017-07-31
4   2017-07-24
Name: Review_Date, dtype: datetime64[ns]


In [68]:
# Kullanıcıların yıl ve ay üzerinden filtreleme yapabilmesini sağlayacağım.
# Değerlendirme yılı ve değerlendirme ayı şeklinde sütunlar oluşturalım.
df['Review_Year']=df['Review_Date'].dt.year

# Eğer kullanıcılar ayları sayısal olarak değil de isim olarak görsün.
df['Review_Month_Name'] = df['Review_Date'].dt.strftime('%B')
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,Country,Review_Year,Review_Month_Name
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Netherlands,2017,August
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Netherlands,2017,August
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",Netherlands,2017,July
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",Netherlands,2017,July
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",Netherlands,2017,July


### Streamlit kısmı.Sonraki aşamalarda devam edeceğim.

### Streamlit widget'ları ile yıl ve ay seçimi
selected_year = st.selectbox('Yıl Seçin', df['Year'].unique())
selected_month = st.selectbox('Ay Seçin', df[df['Year'] == selected_year]['Month'].unique())

### Seçilen yıl ve ay için veri filtreleme
filtered_data = df[(df['Year'] == selected_year) & (df['Month'] == selected_month)]

### Kullanıcıya filtrelenmiş veriyi gösterme
st.write(filtered_data)

In [69]:
# Ayların dağılımını kontrol et
print(df['Review_Month_Name'].value_counts())


Review_Month_Name
August       50615
July         49267
May          46457
October      43773
April        42857
June         42856
September    42341
January      42121
March        41075
December     39574
February     38337
November     35939
Name: count, dtype: int64


In [70]:
# Her yorum,tarih bilgisine göre bir mevsime atansın.

# Mevsim fonksiyonu
def season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    elif month in [12, 1, 2]:
        return 'Winter'
    else:
        return "Unknown!"

# Şimdi season sütunu ekleyelim.
df['Season'] = df['Review_Date'].dt.month.apply(season)

# Sonuçları kontrol edelim.
print(df[['Review_Date','Review_Month_Name', 'Season']].head())

  Review_Date Review_Month_Name  Season
0  2017-08-03            August  Summer
1  2017-08-03            August  Summer
2  2017-07-31              July  Summer
3  2017-07-31              July  Summer
4  2017-07-24              July  Summer


In [71]:
# Dataset'te mevsimlerin oransal dağılımı.
df['Season'].value_counts(normalize=True)

Season
Summer    0.277047
Spring    0.253078
Fall      0.236899
Winter    0.232976
Name: proportion, dtype: float64

Numerik sütunların analizi.

Reviewer_Score: Bu sütun otellerin genel puanlarını temsil ediyor ve model için güçlü bir tahmin edici olabilir.Burada skorları 
düşük-orta-yüksek şeklinde aralıklara bölerek kategorik bir sütun oluşturalım.

In [72]:
# Reviewer_Score değerlerini 0-10 arasında 3 eşit parçaya ayıralım.

bins = [0, 3.33, 6.66, 10]  # Aralıklar
labels = ['Low', 'Medium', 'High']  # Etiketler

#include_lowest=True ile belirtilen ilk aralık(0-3.33)dahil edilir.Medium ->low kategorisinde olur.
df['Reviewer_Score_Range'] = pd.cut(df['Reviewer_Score'], bins=bins, labels=labels, include_lowest=True)

# Oluşturulan sütunu kontrol edelim.
print(df[['Reviewer_Score', 'Reviewer_Score_Range']].head())


   Reviewer_Score Reviewer_Score_Range
0             2.9                  Low
1             7.5                 High
2             7.1                 High
3             3.8               Medium
4             6.7                 High


Kullanıcı puanı->   Reviewer Score

Otelin genel puanı -> Average Score

Şimdi Score_Difference adında bu iki score'un farkını gösteren bir sütun oluşturalım.Bu sütun şu işlevleri yerine getirecek:
Otelin genel puanı yüksek ama kullanıcılar düşük puan veriyorsa otel için bu bir uyarı olabilir.
Genel puan düşük, ancak kullanıcılar yüksek puan veriyorsa otelin imajını iyileştirme potansiyelini gösterebilir.

In [81]:
df['Score_Difference']=df['Reviewer_Score']-df['Average_Score']
print(df[['Reviewer_Score','Average_Score','Score_Difference']])


        Reviewer_Score  Average_Score  Score_Difference
0                  2.9            7.7              -4.8
1                  7.5            7.7              -0.2
2                  7.1            7.7              -0.6
3                  3.8            7.7              -3.9
4                  6.7            7.7              -1.0
...                ...            ...               ...
515733             7.0            8.1              -1.1
515734             5.8            8.1              -2.3
515735             2.5            8.1              -5.6
515736             8.8            8.1               0.7
515737             8.3            8.1               0.2

[515212 rows x 3 columns]


Score_Difference ->Streamlit ile arayuze eklenecek kısım.

import plotly.express as px

## Interaktif bir bar grafiği oluştur
fig = px.bar(df, x=df.index, y='Score_Difference', color='Score_Difference', 
             title='Reviewer Score ve Average Score Farkları', labels={'Score_Difference': 'Fark'})

## Streamlit ile gösterelim
st.plotly_chart(fig)